In [1]:
import torch
import json
from transformers import SiglipProcessor, SiglipModel
from sklearn.metrics.pairwise import cosine_similarity
import os
import numpy as np

c:\Users\Adam\miniconda3\envs\main_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# Modell és processor betöltése
model = SiglipModel.from_pretrained("google/siglip-base-patch16-224").to(device)
processor = SiglipProcessor.from_pretrained("google/siglip-base-patch16-224")

# Előre kiszámolt embeddingek és kép információk betöltése
embeddings = torch.load("C:/Users/Adam/Desktop/applied_ml/dataset/image_embeddings_siglip.pt")
infos = torch.load("C:/Users/Adam/Desktop/applied_ml/dataset/image_info_siglip.pt")

# Képcsoportokról szóló szövegek betöltése
#with open("C:/Users/Adam/Desktop/applied_ml/custom_captions.json", encoding="utf-8") as f:
 #   class_texts = json.load(f)

C:\Users\Adam\AppData\Local\Temp\ipykernel_6472\1860290231.py:8: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load("C:/Users/Adam/Desktop/applied_ml/data

In [4]:
# Szótár a képosztályokhoz tartozó képek és azok embeddingjeihez
class_to_embeddings = {}

for idx, info in enumerate(infos):
    label = str(info['label'])
    if label not in class_to_embeddings:
        class_to_embeddings[label] = []
    class_to_embeddings[label].append(embeddings[idx])

In [ ]:
# Kiszámoljuk az osztályok közötti átlagos cosine similarity-t
similarities_per_class = {}

for class_id, class_embeddings in class_to_embeddings.items():
    embeddings_tensor = torch.stack(class_embeddings)
    
    # Kiszámítjuk a cosine similarity mátrixot az osztályon belül
    similarity_matrix = cosine_similarity(embeddings_tensor.cpu().numpy())
    
    # Az összes párra átlagos cosine similarity-t számolunk
    upper_triangle_indices = np.triu_indices(similarity_matrix.shape[0], k=1)
    
    avg_similarity = similarity_matrix[upper_triangle_indices].mean()
    similarities_per_class[class_id] = avg_similarity.item()  # Konvertáljuk float32 -> float64

# Eredmény mentése fájlba
output_file = "C:/Users/Adam/Desktop/applied_ml/intra_class_similarities.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(similarities_per_class, f, indent=4)

print(f"✅ A cosine similarity-k átlagát sikeresen elmentettük a következő fájlba: {output_file}")

✅ A cosine similarity-k átlagát sikeresen elmentettük a következő fájlba: C:/Users/Adam/Desktop/applied_ml/intra_class_similarities.json


C:\Users\Adam\AppData\Local\Temp\ipykernel_11004\1833923895.py:15: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load("C:/Users/Adam/Desktop/applied_ml/da

In [ ]:
# Képosztályok közötti inter cosine similarity kiszámítása
inter_class_similarities = {}

class_ids = list(class_to_embeddings.keys())

for class_id_1 in class_ids:
    embeddings_class_1 = torch.stack(class_to_embeddings[class_id_1]).cpu().numpy()
    inter_class_similarities[class_id_1] = {}  # Az osztályhoz egy új szótárat hozunk létre
    
    for class_id_2 in class_ids:
        embeddings_class_2 = torch.stack(class_to_embeddings[class_id_2]).cpu().numpy()
        
        # Cosine similarity a két osztály között
        similarity_matrix = cosine_similarity(embeddings_class_1, embeddings_class_2)
        
        # Az átlagos cosine similarity a két osztály között
        avg_similarity = similarity_matrix.mean()
        
        # Az eredmény elmentése (float32 -> float64)
        inter_class_similarities[class_id_1][class_id_2] = float(avg_similarity)

# Eredmény mentése fájlba
output_file = "C:/Users/Adam/Desktop/applied_ml/inter_class_similarities.json"
with open(output_file, "w", encoding="utf-8") as f:
    json.dump(inter_class_similarities, f, indent=4)

print(f"✅ A képosztályok közötti cosine similarity-k sikeresen elmentve a következő fájlba: {output_file}")

C:\Users\Adam\AppData\Local\Temp\ipykernel_11004\1974031112.py:13: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load("C:/Users/Adam/Desktop/applied_ml/da

✅ A képosztályok közötti cosine similarity-k sikeresen elmentve a következő fájlba: C:/Users/Adam/Desktop/applied_ml/inter_class_similarities.json
